In [ ]:
####01_data_splitting#######
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from pathlib import Path 
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"
# --- 配置参数 ---
file_path = DATA_DIR / "JM.xlsx"
target_column = 'Rowing distance'
test_set_size = 0.2  # 20% 作为最终测试集
random_seed = 42     # 设置随机种子以保证结果可复现
num_bins_for_stratification = 4 # 用于分层的箱子数量，可以根据数据分布调整 (例如 3, 4, 或 5)

# --- 从文件路径中提取目录，用于保存输出文件 ---
dev_set_filename = DATA_DIR / "development_set.xlsx"
test_set_filename = DATA_DIR /  "final_test_set.xlsx"

# --- 1. 加载数据 ---
try:
    original_data_df = pd.read_excel(file_path)
    print(f"原始数据集加载成功，总样本数: {len(original_data_df)}")
    if target_column not in original_data_df.columns:
        raise ValueError(f"错误: 目标变量 '{target_column}' 不在提供的Excel文件中。请检查列名。")
except FileNotFoundError:
    print(f"错误: 文件未找到 {file_path}")
    exit()
except Exception as e:
    print(f"加载Excel文件时发生错误: {e}")
    exit()

# --- 2. 为目标变量创建分箱以进行分层抽样 ---
#    我们使用 pd.qcut 来尝试创建等频率的箱子（每个箱子样本数尽量接近）
#    duplicates='drop' 会在有重复边界值时合并箱子，这对于某些分布的数据是必要的
try:
    original_data_df['target_bins'] = pd.qcut(original_data_df[target_column],
                                              q=num_bins_for_stratification,
                                              labels=False,
                                              duplicates='drop')
    print(f"已为目标变量 '{target_column}' 创建 {original_data_df['target_bins'].nunique()} 个分箱用于分层。")
except ValueError as e:
    print(f"警告: 使用 pd.qcut 创建 {num_bins_for_stratification} 个分箱失败: {e}")
    print("这可能是因为数据点在某些值上过于集中，导致无法形成所需数量的唯一分箱边界。")
    print("尝试减少 num_bins_for_stratification 的数量，或者如果数据允许，可以考虑使用 pd.cut 进行等宽分箱。")
    print("为了继续，将不进行分层抽样，而是进行简单的随机抽样。如果分层至关重要，请调整分箱策略。")
    # 如果qcut失败，回退到简单随机抽样，或者你可以选择停止脚本并调整分箱
    use_stratify = False
    stratify_column_data = None
except Exception as e:
    print(f"为目标变量创建分箱时发生未知错误: {e}")
    exit()
else:
    use_stratify = True
    stratify_column_data = original_data_df['target_bins']


# --- 3. 执行数据划分 ---
if use_stratify and stratify_column_data is not None:
    print(f"正在基于 '{target_column}' 的分箱进行分层随机抽样...")
    development_df, final_test_df = train_test_split(
        original_data_df,
        test_size=test_set_size,
        stratify=stratify_column_data, # 基于目标变量的分箱进行分层
        random_state=random_seed
    )
else:
    print("警告: 无法进行分层抽样，将执行简单的随机抽样。")
    development_df, final_test_df = train_test_split(
        original_data_df,
        test_size=test_set_size,
        random_state=random_seed
    )

# --- 4. （可选）删除划分数据后不再需要的辅助分箱列 ---
#    原始DataFrame中的 'target_bins' 列在划分后的df中仍然存在，我们可以选择删除它。
if 'target_bins' in development_df.columns:
    development_df = development_df.drop(columns=['target_bins'])
if 'target_bins' in final_test_df.columns:
    final_test_df = final_test_df.drop(columns=['target_bins'])
if 'target_bins' in original_data_df.columns: # 也从原始副本中删除，以防后续意外使用
    original_data_df = original_data_df.drop(columns=['target_bins'])


# --- 5. 打印划分结果信息 ---
print("\n--- 数据划分结果 ---")
print(f"开发集 (Development Set) 样本数: {len(development_df)} ({len(development_df)/len(original_data_df)*100:.2f}%)")
print(f"最终测试集 (Final Hold-Out Test Set) 样本数: {len(final_test_df)} ({len(final_test_df)/len(original_data_df)*100:.2f}%)")

# 检查目标变量在各集合中的分布（均值和标准差）作为快速校验
print("\n目标变量在各集合中的分布概览:")
print(f"原始数据集中 '{target_column}' 均值: {original_data_df[target_column].mean():.2f}, 标准差: {original_data_df[target_column].std():.2f}")
print(f"开发集中 '{target_column}' 均值: {development_df[target_column].mean():.2f}, 标准差: {development_df[target_column].std():.2f}")
print(f"最终测试集中 '{target_column}' 均值: {final_test_df[target_column].mean():.2f}, 标准差: {final_test_df[target_column].std():.2f}")


# --- 6. 保存划分好的数据集到Excel文件 ---
try:
    development_df.to_excel(dev_set_filename, index=False)
    print(f"\n开发集已成功保存到: {dev_set_filename}")
    final_test_df.to_excel(test_set_filename, index=False)
    print(f"最终测试集已成功保存到: {test_set_filename}")
except Exception as e:
    print(f"\n保存文件时发生错误: {e}")

print("\n代码执行完毕。")

In [ ]:
#####